In [13]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Скрыть/показать код</button>''', raw=True)

Скрыть/показать код

In [8]:
import vk_api
import datetime
from dateutil import parser
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import time
#графики
import matplotlib.pyplot as plt
import plotly.tools as tls
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
from plotly import graph_objs as go
init_notebook_mode(connected = True)
# Decision Tree Classifier
from sklearn import datasets
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

server = 'MARKTSERV\MARKTMSSQL'
database = 'Аналитика_3.0 (serv)'
username = 'ТарасовГВ'
password = 'Tarasov0717'
driver= '{ODBC Driver 13 for SQL Server}'
conn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
#2019
df1=pd.read_sql_query('''SELECT B.[Дата гарантии-заявки(Б)]
  			  ,sum([Сумма БГ(Б)]) as 'Сумма БГ'
			  ,count([Сумма БГ(Б)]) as 'Кол-во БГ'
		  FROM [Аналитика_3.0 (serv)].[dbo].[Банки] B
		  where [id.сп_Статусы] in (17,20,31,33) and [Дата гарантии-заявки(Б)]>='2019-01-01'
		  group by B.[Дата гарантии-заявки(Б)]''', conn, index_col='Дата гарантии-заявки(Б)').sort_index() 
#2018
df2=pd.read_sql_query('''SELECT DATEADD(year,1,B.[Дата гарантии-заявки(Б)]) as 'Дата гарантии-заявки(Б)'
  			  ,sum([Сумма БГ(Б)]) as 'Сумма БГ'
			  ,count([Сумма БГ(Б)]) as 'Кол-во БГ'
		  FROM [Аналитика_3.0 (serv)].[dbo].[Банки] B
		  where [id.сп_Статусы] in (17,20,31,33) and [Дата гарантии-заявки(Б)]>='2018-01-01' and [Дата гарантии-заявки(Б)]<'2019-01-01'
		  group by B.[Дата гарантии-заявки(Б)]''', conn, index_col='Дата гарантии-заявки(Б)').sort_index() 
#df1=df01.sort_values(['Дата гарантии-заявки(Б)']).reset_index(drop = True)  
#df2=df02.sort_values(['Дата гарантии-заявки(Б)']).reset_index(drop = True)

ser_sum_1 = (df1['Сумма БГ']/1000000).round(0)
ser_sum_2 = (df2['Сумма БГ']/1000000).round(0)
ser_kol_1 = df1['Кол-во БГ']
ser_kol_2 = df2['Кол-во БГ']

data_sum_1 = go.Scatter(
        x=ser_sum_1.index, # assign x as the dataframe column 'x'
        y=ser_sum_1,
        mode='lines+markers+text',
        text=ser_sum_1,
        textposition="top center",
        line=go.scatter.Line(color="blue"),
        name='sum_2019'        
    )
data_sum_2 = go.Scatter(
        x=ser_sum_2.index, # assign x as the dataframe column 'x'
        y=ser_sum_2,
        mode='lines+markers+text',
        line=go.scatter.Line(color="lightcoral"),
        name='sum_2018'        
    )
data_kol_1 = go.Scatter(
        x=ser_kol_1.index, # assign x as the dataframe column 'x'
        y=ser_kol_1,
        mode='lines+markers+text',
        text=ser_kol_1,
        textposition="top center",
        line=go.scatter.Line(color="darkblue"),
        name='kol_2019'        
    )
data_kol_2 = go.Scatter(
        x=ser_kol_2.index, # assign x as the dataframe column 'x'
        y=ser_kol_2,
        mode='lines+markers+text',
        line=go.scatter.Line(color="lightsalmon"),
        name='kol_2018',       
    )
subplot_titles=("Сумма БГ, млн руб", "Кол-во БГ, шт")
fig = make_subplots(rows=1, cols=2, subplot_titles=subplot_titles)
fig.add_trace(data_sum_1, row=1, col=1)
fig.add_trace(data_sum_2, row=1, col=1)
fig.add_trace(data_kol_1, row=1, col=2)
fig.add_trace(data_kol_2, row=1, col=2)
fig.update_layout(title_text="Totall продажи ServiceHub",title_font_size=50,title_x=0.5, # заголовок и ниже оси
                  xaxis_title_text="Дата гарантии-заявки",xaxis_title_font_size=20,xaxis_title_font_family="Courier New, monospace",xaxis_title_font_color="#7f7f7f",
                  yaxis_title_text="Сумма БГ, млн руб",yaxis_title_font_size=20,yaxis_title_font_family="Courier New, monospace",yaxis_title_font_color="#7f7f7f")
fig.update_xaxes(title_text="Дата гарантии-заявки", title_font_size=20,title_font_family="Courier New, monospace",title_font_color="#7f7f7f", row=1, col=2)
fig.update_yaxes(title_text="Кол-во БГ, шт", title_font_size=20,title_font_family="Courier New, monospace",title_font_color="#7f7f7f", row=1, col=2)
fig.layout.font.family = ' Rockwell '
for i in fig['layout']['annotations']:
    i['font'] = dict(size=30,color='#ff0000')
plot(fig, filename='vidano.html', auto_open=False)
fig.show()

def plotMovingAverage(series, n):
    rolling_mean = series.rolling(window=n).mean()
    # При желании, можно строить и доверительные интервалы для сглаженных значений
    rolling_std =  series.rolling(window=n).std()
    upper_bond = rolling_mean+1.96*rolling_std  # надежностью γ = 0.95;  Ф(tkp) = γ/2 = 0.95/2 = 0.475; По таблице функции Лапласа найдем, при каком tkp значение Ф(tkp) = 0.475
    lower_bond = rolling_mean-1.96*rolling_std  # Tkp(γ) = (0.475) = 1.96

    figM=plt.figure(figsize=(25,12))
    plt.title("Moving average\n window size = {}".format(n))
    plt.plot(rolling_mean, "g", label="Rolling mean trend")
    plt.plot(upper_bond, "r--", label="Upper Bond")
    plt.plot(lower_bond, "r--", label="Lower Bond")
    plt.plot(series[n:], label="Actual values")
    plt.legend(loc="upper left")
    plt.grid(True)
    plotly_fig = tls.mpl_to_plotly(figM)
    plotly_fig.update_layout(title_text="График",title_font_size=50,title_x=0.5, # заголовок и ниже оси
                  xaxis_title_text="Дата гарантии-заявки",xaxis_title_font_size=20,xaxis_title_font_family="Courier New, monospace",xaxis_title_font_color="#7f7f7f",
                  yaxis_title_text="Сумма БГ, руб",yaxis_title_font_size=20,yaxis_title_font_family="Courier New, monospace",yaxis_title_font_color="#7f7f7f")

    plot(plotly_fig, filename='plot_average_'+str(n)+'.html')
#plotMovingAverage(ser1, 7) # сглаживаем по дням
#plotMovingAverage(ser1, 30) # сглаживаем по неделям